In [5]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras import Model 
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from scipy.special import kl_div

import random

random.seed(20240528)
import tensorflow_ranking as tfr 
import time 

# Section 1: Helper functions and set-ups 

In [4]:
## Graphing set-up 
import seaborn as sns
x = np.linspace(-4, 4, 100)
tencent_blue = (0,0.3215686274509804,0.8509803921568627)
tencent_orange = (0.9333333333333333, 0.49411764705882355, 0.2784313725490196)


# Calculate y-values for the standard normal density curve
y_standard_normal = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * x**2)

In [2]:
## True Exposure Model and Data Generating Process 
def logistic_row(row):
    return np.exp(row) / np.sum(np.exp(row))

def dim_est(obs_T, obs_C, treated_probability, Q):
    n1,n0 = len(obs_T), len(obs_C)
    tau1 = np.sum(obs_T) / (Q*treated_probability)
    tau0 = np.sum(obs_C)/(Q * (1-treated_probability))
    estimate = tau1 - tau0
    # var = np.sum((obs_T - tau1)**2)/ Q/treated_probability**2  + np.sum((obs_C - tau0)**2)/ Q/(1-treated_probability)**2 
    var = (np.sum((obs_T / treated_probability - estimate) ** 2) + np.sum((obs_C / (1-treated_probability) - estimate) ** 2)) / Q
    return estimate, var

def DGP_new_heterogeneous(J, Q, K, promo_ratio, query_matrix, X_goodbads, X_utility, treated_probability=0.5, treat_control_pool = [True, False]):
    ## Randomize over the treatment assignment matrix 
    treatment_dict = {}
    for j in range(J):
        treatment_dict[j] = np.random.choice(treat_control_pool, 1, p=[treated_probability, 1 - treated_probability])

    W_matrix = []
    for each_query in range(Q):
         W_matrix = np.append(W_matrix, [treatment_dict[ind] for ind in query_matrix[each_query]])

    outcome_noise =  np.random.normal(size=(Q, K)) 
    W_matrix = W_matrix.reshape(Q,K)
    # W_matrix = W_matrix.reshape(Q,K)
    final_score_matrix = W_matrix * promo_ratio * X_goodbads   + X_utility

    X_logit = np.apply_along_axis(logistic_row, axis=1, arr=final_score_matrix)
    expose_indices = np.array([np.random.choice(np.arange(K), size = 1, p = X_logit[i,:]) for i in range(Q)])
    inddds = np.array(list(np.arange(K)) * Q).reshape(Q,K)
    exposure_matrix = np.array([inddds[i,:] == expose_indices[i] for i in range(Q)])

    ## Outcome model  
    ## First: a true outcome model of Exponential 
    outcome_potential = X_utility

    return query_matrix, X_goodbads, X_utility,W_matrix, exposure_matrix, outcome_potential, X_logit

In [26]:
## Helper functions 

def debias_estimator_new(Hfuncs, debias_terms,tau_hat):
    psi_functions = Hfuncs - debias_terms 
    undebiased_estimator = np.mean(Hfuncs)
    debiased_estimator = np.mean(psi_functions)
    variance_estimator = np.sum((psi_functions - tau_hat)**2) /len(psi_functions)
    return debiased_estimator, variance_estimator, undebiased_estimator 


def undebias_estimator_new(Hfuncs,tau_hat):
    psi_functions = Hfuncs 
    undebiased_estimator = np.mean(Hfuncs)
    variance_estimator = np.sum((psi_functions - tau_hat)**2) /len(psi_functions)
    
    return undebiased_estimator, variance_estimator 


def is_invertible(matrix):
    return np.linalg.det(matrix) != 0


    
def permute_treatment_dict(J, L):
    perm_dict = {}
    for j in range(J):
        perm_dict[j] = np.random.choice(L+1, 1)
    return perm_dict

## Helper function for cross validation
def generate_indices(n, K):
    ## Split original sample of size n into K sets 
    indices = np.linspace(0, n, K+1, dtype=int)
    return list(zip(indices[:-1], indices[1:]))


def train_test_split(input_data, all_inds, kth_test):
    
    training_ind = [all_inds[i] for i in range(len(all_inds)) if i != kth_test]
    test_start, test_end = all_inds[kth_test]
    if not tf.is_tensor(input_data):
        training_data = np.concatenate([input_data[elm[0]:elm[1]] for elm in training_ind])
    else:
        
        training_data = tf.concat([input_data[elm[0]:elm[1]] for elm in training_ind], axis = 0)
    testing_data = input_data[test_start:test_end]
    return training_data, testing_data 

In [8]:
## True Model 

class MyModel_True_Heterogeneous:
    def __init__(self, k, num_treats,promo_ratio):

        self.k = k
        self.promo_ratio = promo_ratio
        self.num_treats = num_treats

    def predict(self,inputs):
        Q_input = inputs.shape[0]
        split_structure =  [1]+ [1] + [1] * self.num_treats + [1]
        splitted_elements = tf.split(inputs, split_structure, axis=2)
        X_utility =  np.squeeze(np.array(splitted_elements[1]), axis=2)
        X_goodbads = np.squeeze(np.array(splitted_elements[0]), axis = 2)

        W_matrix =  np.squeeze(np.array(splitted_elements[2]), axis =2 )

        final_score_matrix =  W_matrix * self.promo_ratio * X_goodbads + X_utility

        ## First element of each row 
        first_elm = X_utility[:,0]
        minus_matrix = first_elm.reshape((len(first_elm),1))@np.ones((1,K))
        final_score_matrix_normalized = final_score_matrix - minus_matrix
        ## Correct exposure probability 
        X_logit = np.apply_along_axis(logistic_row, axis=1, arr=final_score_matrix_normalized)
    
        # expose_indices = np.argmax(X_logit, axis = 1)
        # inddds = np.array(list(np.arange(K)) * Q_input).reshape(Q_input,K)
        # exposure_matrix = np.array([inddds[i,:] == expose_indices[i] for i in range(Q_input)])

        ## Outcome model  
        
        ## First: a true outcome model of Exponential 
        outcome_potential = X_utility
        pred_out = np.sum(X_logit * outcome_potential, axis = 1 )
        pred_out = pred_out.reshape(pred_out.shape[0], 1 )
        return np.concatenate([X_logit, pred_out], axis = 1 )
    
class MyModel_True_Homogeneous:
    def __init__(self, k, num_treats,promo_ratio):

        self.k = k
        self.promo_ratio = promo_ratio
        self.num_treats = num_treats

    def predict(self,inputs):
        Q_input = inputs.shape[0]
        split_structure =  [1]+ [1] + [1] * self.num_treats + [1]
        splitted_elements = tf.split(inputs, split_structure, axis=2)
        X_utility =  np.squeeze(np.array(splitted_elements[1]), axis=2)
        X_goodbads = np.squeeze(np.array(splitted_elements[0]), axis = 2)

        W_matrix =  np.squeeze(np.array(splitted_elements[2]), axis =2 )

        final_score_matrix = W_matrix * self.promo_ratio + X_utility

        ## First element of each row 
        first_elm = X_utility[:,0]
        minus_matrix = first_elm.reshape((len(first_elm),1))@np.ones((1,K))
        final_score_matrix_normalized = final_score_matrix - minus_matrix
        ## Correct exposure probability 
        X_logit = np.apply_along_axis(logistic_row, axis=1, arr=final_score_matrix_normalized)
    
        # expose_indices = np.argmax(X_logit, axis = 1)
        # inddds = np.array(list(np.arange(K)) * Q_input).reshape(Q_input,K)
        # exposure_matrix = np.array([inddds[i,:] == expose_indices[i] for i in range(Q_input)])

        ## Outcome model  
        
        ## First: a true outcome model of Exponential 
        outcome_potential = X_utility
        pred_out = np.sum(X_logit * outcome_potential, axis = 1 )
        pred_out = pred_out.reshape(pred_out.shape[0], 1 )
        return np.concatenate([X_logit, pred_out], axis = 1 )
## True Model 

# class MyModel_Random:
#     def __init__(self, k, num_treats,promo_ratio):

#         self.k = k
#         self.promo_ratio = promo_ratio
#         self.num_treats = num_treats

#     def predict(self,inputs):
#         output_shape = np.array(input_3d_test_treat.shape)[:2]
#         array = np.random.rand(output_shape[0],output_shape[1])
#         # Compute the sum of each row
#         row_sums = np.sum(array, axis=1)

#         # Reshape the row sums to make them compatible for broadcasting
#         row_sums = row_sums.reshape(-1, 1)

#         normalized_array = array / row_sums

#         return normalized_array

# Section 2: Data-generating process

In [13]:
## Number of videos 
J = 30
## Consideration set size 
K = 5
k=5
## Generate some queries along with the recommendation model 
Q = 1200
## Uplift factor 
uplift_factor = 1 

utility_score_matrix = np.exp(np.random.normal(size=(Q,J)))

good_bad_dict = {} 
treatment_dict = {} 
utility_score = {} 
for j in range(J):
    good_bad_dict[j] = np.random.choice([True,False], 1)
    utility_score[j] = np.random.uniform()
X_goodbads = []
X_utility = []
query_matrix = []
for each_query in range(Q):
    ## Form the consideration set 
    selected_indices = np.random.choice(np.arange(J), K, replace= False)
    query_matrix += [selected_indices]
    X_goodbads = np.append(X_goodbads,[good_bad_dict[ind] for ind in selected_indices])
    X_utility = np.append(X_utility, [utility_score_matrix[each_query, ind] for ind in selected_indices])
X_goodbads = X_goodbads.reshape(Q, K)
X_utility = X_utility.reshape(Q, K)
X_utility = X_utility + X_goodbads 

In [24]:
## Find the ground truth 
query_matrix_T,X_goodbads_T,X_utility_T,W_matrix_T, exposure_matrix_T, outcome_potential_T, X_logit_T = DGP_new_heterogeneous(J, Q, K,1, query_matrix, X_goodbads, X_utility,  treat_control_pool = [True, True])
query_matrix_C,X_goodbads_C,X_utility_C,W_matrix_C, exposure_matrix_C, outcome_potential_C, X_logit_C= DGP_new_heterogeneous(J, Q, K, 1, query_matrix, X_goodbads, X_utility,  treat_control_pool = [False, False])
T_gt = np.sum(X_logit_T * X_utility , axis = 1 )
C_gt = np.sum(X_logit_C * X_utility , axis = 1 )

ground_truth = np.mean(T_gt) - np.mean(C_gt)

In [25]:
ground_truth

0.013515599579713822

# Section 3: Simultation 

In [34]:
dim_B, dim_var_B= [],[]
debias_B_true, debias_var_B_true = [],[] 
undebias_B_true, debias_var_old_B_true = [],[]
truth= []
undebias_var_B_true = []
## Number of iterations of DGP
B = 50
## Number of videos 
training_ratio = 0.4
JQ_sizes = [(J,Q)]
Ks = [5]
num_features = 2 



## True Outcome Model test 
L = 1
ith_treat = 0

## Number of iterations for Hessian matrix estimation 
M = 100
groupNames = [0,1]
uplift_ratio = uplift_factor
k = K
n_folds = 3

In [43]:
for (J, Q) in JQ_sizes:
    for K in Ks:
        print("Start K = {}, Q = {}, J = {}".format(str(K), str(Q), str(J)))
        dim_B, dim_var_B= [],[]
        debias_B_true, debias_var_B_true = [],[] 
        undebias_B_true, debias_var_old_B_true = [],[]

        for b in range(B):
            query_matrix, X_goodbads, X_utility,W_matrix, exposure_matrix, outcome_potential, X_logit = DGP_new_heterogeneous(J, Q, K, uplift_ratio, query_matrix, X_goodbads, X_utility, treat_control_pool = [True, False])
            
            ## Cross-fitting indices 
            all_inds = generate_indices(np.array(query_matrix).shape[0], n_folds)

            ## Iterate over each fold for cross-validation. 
            hfuncs_each_fold,  debias_terms_each_fold = {},{}

            for f in range(n_folds):
                f_start, f_end = all_inds[f]
                
                ## Train-test split 
                query_train, query_test = train_test_split(np.array(query_matrix), all_inds, f)
                X_goodbads_train, X_goodbads_test =  train_test_split(X_goodbads, all_inds, f) 
                X_utility_train, X_utility_test =  train_test_split(X_utility, all_inds, f)  
                W_matrix_train, W_matrix_test = train_test_split(W_matrix, all_inds, f) 
                observed_queries_treatment = np.sum(exposure_matrix * W_matrix, axis = 1 )
                observed_outcome = np.sum(outcome_potential * exposure_matrix, axis = 1 )

                T, C = observed_outcome[observed_queries_treatment == groupNames[ith_treat + 1 ]] , observed_outcome[observed_queries_treatment == 0]  
                exposure_matrix_train,exposure_matrix_test =train_test_split(exposure_matrix, all_inds, f) 
                outcome_matrix = exposure_matrix * outcome_potential
                outcome_matrix = np.sum(outcome_matrix, axis = 1 ).reshape(outcome_matrix.shape[0],1)

                observed_outcome_train, observed_outcome_test = train_test_split(observed_outcome, all_inds, f) 
                outcome_matrix_train, outcome_matrix_test = train_test_split(outcome_matrix, all_inds, f) 
                outcome_potential_train, outcome_potential_test = train_test_split(outcome_potential, all_inds, f)  
                inputs_3d_train = tf.stack([X_goodbads_train,X_utility_train, W_matrix_train, X_utility_train ], axis = -1)
                inputs_3d_test = tf.stack([X_goodbads_test,X_utility_test, W_matrix_test, X_utility_test], axis = -1)

                input_3d_test_treat = tf.stack([X_goodbads_test,X_utility_test, np.ones(W_matrix_test.shape), X_utility_test], axis = -1)
                input_3d_test_control = tf.stack([X_goodbads_test,X_utility_test, np.zeros(W_matrix_test.shape), X_utility_test], axis = -1)
                output_3d_train = tf.concat([tf.cast(exposure_matrix_train, dtype=float),outcome_matrix_train], axis = 1)
                output_3d_test = tf.concat([tf.cast(exposure_matrix_test, dtype=float),outcome_matrix_test ], axis = 1)

                exposure_indicator_array = exposure_matrix_test

                ## Get treatment indicator matrix
                w_dict = {}

                for l in range(L):
                    w_dict[l] = tf.convert_to_tensor(W_matrix == groupNames[l+1], dtype = float)

                # training_num = int(W_matrix.shape[0] * training_ratio)
                # testing_num = W_matrix.shape[0] - training_num


                w_all_treat = tf.convert_to_tensor(np.array([[ith_treat + 1] * k for _ in range(W_matrix.shape[0])],dtype='float32'))
                w_all_control = tf.convert_to_tensor(np.array([[0] * k for _ in range(W_matrix.shape[0])],dtype='float32'))

                inputs_all_treat_3d = tf.stack([X_goodbads,X_utility] + [w_all_treat if l == ith_treat else w_all_control for l in range(L)] +[ X_utility], axis = 2)
                inputs_all_control_3d = tf.stack([X_goodbads,X_utility] + [w_all_control if l == ith_treat else w_all_control for l in range(L)] +[ X_utility ], axis = 2)
                inputs_all_treat_3d = tf.cast(inputs_all_treat_3d, dtype = 'float32')
                inputs_all_control_3d = tf.cast(inputs_all_control_3d, dtype = 'float32')


                ## All other all_treated 
                inputs_all_treat_3d_dict = {} 
                for l in range(L):
                    inputs_all_treat_3d_l = tf.stack([X_goodbads,X_utility] + [w_all_treat if l == v else w_all_control for v in range(L)] +[ X_utility ], axis = 2)
                    #inputs_all_treat_3d_l = tf.stack([x_basebid, x_sort_score, x_bid,x_ecpm, x_cvr] + [w_all_treat if v == l else w_all_control for v in range(L)] + [x_cvr], axis = 2)
                    inputs_all_treat_3d_dict[l] = tf.cast(inputs_all_treat_3d_l, dtype = 'float32')

                exposure_indicator_outcome_train, exposure_indicator_outcome_test = outcome_matrix_train, outcome_matrix_test
                inputs_all_treat_3d_test = input_3d_test_treat
                inputs_all_control_3d_test = input_3d_test_control
                is_selected_indicator_train,is_selected_indicator_test = exposure_matrix_train,exposure_matrix_test

                treat_control_dict = {} 
                for l in range(L):

                    
                    inputs_3d_train_l,inputs_3d_test_l= train_test_split(inputs_all_treat_3d_dict[l], all_inds, f) 

                    treat_control_dict[l] = {'train':inputs_3d_train_l, 'test': inputs_3d_test_l}

                myModelMultiple = MyModel_True_Heterogeneous(K, L, uplift_ratio)
                # myModelMultiple_random = MyModel_Random(K, L, uplift_ratio)
                # myModelMultiple.compile(loss=custom_loss,optimizer=tf.keras.optimizers.legacy.Adam())
                # myModelMultiple.fit(input_3d_train,output_3d_train , epochs=10, verbose=False)
                exposure_indicator_array = is_selected_indicator_test
                treatment_indicator_array = 1 * (np.array(w_dict[ith_treat])[f_start:f_end,:])
                res_tempt = np.array(myModelMultiple.predict(inputs_all_treat_3d_test)) - np.array(myModelMultiple.predict(inputs_all_control_3d_test))


                # pred_H_new = np.array(myModelMultiple.predict(inputs_all_treat_3d_test)) - np.array(myModelMultiple.predict(inputs_all_control_3d_test))
                model_pred_H = np.array(myModelMultiple.predict(inputs_3d_test))
                model_pred_all_treat = myModelMultiple.predict(inputs_all_treat_3d_test)
                model_pred_all_control = myModelMultiple.predict(inputs_all_control_3d_test)
                all_treat_array, all_control_array = np.array(model_pred_all_treat), np.array(model_pred_all_control)

                ## All other counterfactuals 
                counterfactual_pred_dict = {} 
                for l in range(L):
                    counterfactual_pred_dict[l] = myModelMultiple.predict(treat_control_dict[l]['test'])

                ## Outcome - prediction model 
                indicator_bool = tf.cast(is_selected_indicator_train, dtype=tf.bool)
                selected_elements = tf.boolean_mask(inputs_3d_train[:,:,:num_features], indicator_bool)

                input_to_outcomemodel_train = tf.reshape(selected_elements, (inputs_3d_train.shape[0], num_features))
                # Define your base model
                base_model = tf.keras.Sequential()
                base_model.add(layers.Dense(1, input_shape=(num_features,)))

                # Compile the model
                base_model.compile(optimizer='sgd', loss='mean_squared_error')
                base_model.fit(input_to_outcomemodel_train,output_3d_train[:, K],epochs=50, verbose=False)
                # Now define a new model for prediction
                model_for_prediction = tf.keras.Sequential()
                model_for_prediction.add(layers.TimeDistributed(base_model, input_shape=(K, num_features)))
                predictions = model_for_prediction.predict(inputs_3d_test[:,:,:num_features])
                # Remove the third dimension of size 1
                numpy_array_pred = np.squeeze(predictions, axis=2)

                mus_T, mus_C  = numpy_array_pred,numpy_array_pred
                p_T, p_C  = all_treat_array[:, :k], all_control_array[:,:k]
                rewards_array = observed_outcome_test
                rewards_array = rewards_array.reshape(rewards_array.shape[0],1)
                Ey1,Ey0 = np.sum(mus_T * p_T, axis = 1), np.sum(mus_C * p_C, axis = 1)
                pv1,pv0 = np.sum(exposure_indicator_array * p_T, axis = 1), np.sum(exposure_indicator_array * p_C, axis = 1)

                pv_given_uvw = p_T * treatment_indicator_array + p_C * (1 - treatment_indicator_array)


                p_realized = model_pred_H[:,:K]



                ## 1. COMPUTE THE GRADIENT OF LOSSS  
                ## FIX: change to realized outcome 
                #dl1dtheta0 = pv_given_uvw - exposure_indicator_array
                dl1dtheta0 = p_realized - exposure_indicator_array
                dl1dtheta0 = dl1dtheta0[:, 1:] 


                ## FIX: iterate over all L 
                dl1dthetal_dict = {} 
                for l in range(L):
                    treatment_indicator_array_l = w_dict[l][f_start:f_end, :]
                    dl1dthetal_dict[l] = treatment_indicator_array_l *  (p_realized - exposure_indicator_array)
                dl2dmu = exposure_indicator_array * (mus_T -rewards_array)
                gradient_vector_l = np.concatenate([dl1dtheta0]+[dl1dthetal_dict[l] for l in range(L)] +[dl2dmu], axis =1 )





                ## 2. COMPUTE  THE GRADIENT OF H FUNCTION
                dHdtheta0 = p_T * (mus_T - Ey1.reshape(mus_T.shape[0],1)) - p_C * (mus_C - Ey0.reshape(mus_C.shape[0],1))
                dHdtheta0 = dHdtheta0[:, 1:]



                ## FIX: iterate over each l 
                dHdthetal_dict = {} 
                for l in range(L):

                    p_T_thetal = counterfactual_pred_dict[l][:,:k]
                    Eyl = np.sum(mus_T * p_T_thetal, axis = 1)
                    dHdthetal_dict[l] = p_T_thetal * (mus_T - Eyl.reshape(mus_T.shape[0],1))
                    ## 0 for the groups that are not the target treatment group 
                    if l != ith_treat:
                        dHdthetal_dict[l] = 0 * (p_T_thetal * (mus_T - Eyl.reshape(mus_T.shape[0],1)))

                #dHdthetal = p_T * (mus_T - Ey1.reshape(mus_T.shape[0],1))
                dHdmu = p_T - p_C
                #gradient_vector_H = np.concatenate([dHdtheta0,dHdthetal,dHdmu], axis =1 )

                ## FIX: iterate over all l 
                gradient_vector_H = np.concatenate([dHdtheta0]+[dHdthetal_dict[l] for l in range(L)]+[dHdmu], axis =1 )
                ## Gradient over all other treatments 





                ## 3. FIND THE EXPECTATION OF HESSIAN MATRIX 



                Hessian_all = np.zeros((inputs_3d_test.shape[0],(L+2) * K - 1,  (L+2) * K - 1))

                montecarlo_expected_probability = np.zeros(exposure_indicator_array.shape)

                selected_indicator_dict  = {}
                assignment_pd_dict = {} 
                dmu_dict = {} 




                # for m in range(M):
                #     treat_dict_m = permute_treatment_dict(J)





                M = 500
                for m in range(M):
                    w_dict_m = {} 
                    treat_dict_m = permute_treatment_dict(J, L + 1)
                    W_matrix_m = []
                    for i in range(np.array(query_matrix).shape[0]):
                        ## Form the consideration set 
                        each_query=np.array(query_matrix)[i,:]
                        W_matrix_m = np.append(W_matrix_m, [[treat_dict_m[ind] for ind in each_query]])

                    W_matrix_m = W_matrix_m.reshape(np.array(query_matrix).shape)



                    for l in range(L):
                        w_dict_m[l] = tf.convert_to_tensor(W_matrix_m == groupNames[l + 1], dtype = float)


                    inputs_3d_m = tf.stack([X_goodbads,X_utility]+  [w_dict_m[l] for l in range(L)] +[X_goodbads], axis = -1)
                    inputs_3d_test_m = inputs_3d_m[f_start:f_end,:]
                    model_pred_m = np.array(myModelMultiple.predict(inputs_3d_test_m))[:,:k]
                    outer_product_pv1pv2 = np.array([np.outer(row_[1:], row_[1:]) for row_ in model_pred_m])
                    outer_product_treatment_indicator = np.array([np.outer(row_, row_) for row_ in model_pred_m])
                    outer_product_pv1_one_minus_pv2 = np.array([np.outer(row_, 1-row_) for row_ in model_pred_m])
                    # p_treat = 1/(L+1) 

                    is_selected_indicator_test = np.array(exposure_matrix[f_start:f_end,:])
                    selected_indicator_dict[m] = is_selected_indicator_test 
                    d2l2dtheta0 = - np.array([np.outer(row_, row_) for row_ in model_pred_m])

                    ## FIX: Iterate over l 
                    d2l2dthetal_dict = {}
                    for l in range(L):
                        ## K by K 

                        w_m_l = np.array(w_dict_m[l][f_start:f_end,:])

                        ## Off-diagonal terms 
                        d2l2dtheta1 =  np.array([np.outer(row_, row_) for row_ in w_m_l]) * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                        # d2l2dtheta1 = - p_treat * p_treat * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                        ## Modify diagonal terms
                        for i in range(d2l2dtheta1.shape[0]):
                            treat_indicator_i = w_m_l[i,:]
                            probs_i = model_pred_m[i,:]
                            np.fill_diagonal(d2l2dtheta1[i],treat_indicator_i * probs_i * (1-probs_i))
                            # np.fill_diagonal(d2l2dtheta1[i], p_treat * probs_i * (1-probs_i))
                        d2l2dthetal_dict[l] = d2l2dtheta1



                    ## FIX: iterate over all l1, l2 
                    d2ldthetal1dthetal2 = {} 
                    for l in range(L):
                        w_m_l = np.array(w_dict_m[l][f_start:f_end,:])
                        ## Off-diagonal terms 
                        ## !!!!!!!!!!!!!!!!! Ruohan: please double check the following, I modified the original one.
                        d2l2dtheta0dtheta1 = - np.multiply(w_m_l[:,np.newaxis, :], np.array([np.outer(row_, row_) for row_ in model_pred_m]))
                        # d2l2dtheta0dtheta1 = - p_treat * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                        for i in range(d2l2dtheta0dtheta1.shape[0]):
                            treat_indicator_i = w_m_l[i,:]
                            p_1minusp_i = model_pred_m[i,:] * (1 - model_pred_m[i,:])
                            np.fill_diagonal(d2l2dtheta0dtheta1[i], treat_indicator_i * p_1minusp_i)
                            # np.fill_diagonal(d2l2dtheta0dtheta1[i], p_treat * p_1minusp_i)

                        ## NOTE: -1 to indicate the baseline theta 
                        d2ldthetal1dthetal2[(-1,l)] = d2l2dtheta0dtheta1[:,1:,:]
                        d2ldthetal1dthetal2[(l,-1)] = np.transpose(d2l2dtheta0dtheta1[:,1:,:], (0,2,1))
                        for l_prime in range(L):
                            if l != l_prime: 
                                w_m_l = np.array(w_dict_m[l][training_num:,:])
                                w_m_l_prime = np.array(w_dict_m[l_prime][training_num:,:])
                                indicator_outer = np.array([np.outer(w_m_l[i,:], w_m_l_prime[i,:]) for i in range(w_m_l.shape[0])])

                                d2l2dthetal1dthetal2 = -  indicator_outer * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                                # d2l2dthetal1dthetal2 = -  p_treat * p_treat * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                                d2ldthetal1dthetal2[(l,l_prime)]  = d2l2dthetal1dthetal2
                                d2ldthetal1dthetal2[(l_prime,l)]  = np.transpose(d2l2dthetal1dthetal2, (0,2,1))
                            else:
                                d2ldthetal1dthetal2[(l,l)] = d2l2dthetal_dict[l]


                    d2l2dmu = np.zeros(d2l2dtheta1.shape)

                    # treatment_indicator_array_m = 
                    for i in range(d2l2dmu.shape[0]):
                        #p_1minusp_i = (1 - model_pred_m[i,:]) * (1 - model_pred_m[i,:])
                        p_1minusp_i = model_pred_m[i,:] * (1 - model_pred_m[i,:])
                        # treatment_i = treatment_indicator_array[i,:]
                        # exposure_i = is_selected_indicator_test[i,:]
                        np.fill_diagonal(d2l2dtheta0[i], p_1minusp_i)
                        np.fill_diagonal(d2l2dmu[i], model_pred_m[i,:])



                    d2l2dtheta0 = d2l2dtheta0[:,1:, 1:]
                    # d2l2dtheta01_k_m_1_by_k = d2l2dtheta01[:, 1:,:]
                    # d2l2dtheta10_k_by_k_m_1 = d2l2dtheta01[:, :,1:]
                    Hessian_first_row = np.concatenate([d2l2dtheta0] + [d2ldthetal1dthetal2[(-1, l)] for l in range(L)] + [np.zeros((d2l2dtheta0.shape[0], K-1, K))], axis =2)

                    ## 1 to L + 1 row 
                    Hessian_middle_dict = {}
                    for l in range(L):
                        row_l = np.concatenate([d2ldthetal1dthetal2[(l, -1)]] + [d2ldthetal1dthetal2[(l, l_prime)] for l_prime in range(L)] +[np.zeros((d2l2dtheta0.shape[0], K, K))], axis =2)

                        Hessian_middle_dict[l] = row_l                                                                           


                    Hessian_third_row = np.concatenate((np.zeros((d2l2dtheta0.shape[0], K, K  * (L + 1 ) - 1 )), d2l2dmu), axis =2)

                    Hessian = np.concatenate([Hessian_first_row] + [Hessian_middle_dict[l] for l in range(L)] + [Hessian_third_row], axis = 1 )

                    dmu_dict[m] = d2l2dmu

                    Hessian_all = Hessian_all + Hessian

                Hessian_final = Hessian_all / M
                count_finite = 0
                score_funcs = np.zeros(len(Hessian_final))
                for i in range(len(Hessian_final)):
                    if is_invertible(Hessian_final[i]):
                        try:
                            score_funcs[i] = gradient_vector_H[i]@np.linalg.inv(Hessian_final[i])@gradient_vector_l[i]
                            count_finite += 1 
                        except: 
                            print("Fail for inversion")
                outs_1 = res_tempt[score_funcs !=0,K]


                ## END OF FOR LOOP FOR EACH ITERATION OVER CROSS FITTING
                hfuncs_f, debias_term_f = Ey1 - Ey0,score_funcs
                # debias_point_f,  debias_var_f, undebiased_point_f  = debias_estimator(outs_1, score_funcs[score_funcs!=0])
                # debias_point_each_fold += [debias_point_f]  
                # debias_var_each_fold += [debias_var_f]
                # undebiased_point_each_fold += [undebiased_point_f]
                hfuncs_each_fold[f] =hfuncs_f
                debias_terms_each_fold[f] = debias_term_f

            tau_hat_undebias = np.mean([ np.mean(hfuncs_each_fold[f])for f in range(n_folds)])
            tau_hat_debias = np.mean([ np.mean(hfuncs_each_fold[f] - debias_terms_each_fold[f])  for f in range(n_folds)])
            debias_point = tau_hat_debias
            debias_var = np.mean([debias_estimator_new(hfuncs_each_fold[f] ,debias_terms_each_fold[f], tau_hat_debias)[1] for f in range(n_folds)])
            undebias_var = np.mean([undebias_estimator_new(hfuncs_each_fold[f] ,tau_hat_undebias)[1] for f in range(n_folds)])
            undebias_point = tau_hat_undebias
            dim_point, dim_var = dim_est(T, C)
            dim_B += [dim_point]
            dim_var_B += [dim_var]
            debias_B_true += [debias_point]
            debias_var_B_true += [debias_var]
            undebias_B_true += [undebias_point]
            undebias_var_B_true += [undebias_var]

        result_df = pd.DataFrame({"debias_point": debias_B_true, "debias_var":debias_var_B_true, "dim": dim_B, 
                                 "dim_var":dim_var_B, "undebias_point": undebias_B_true, "undebias_var": undebias_var_B_true, "J" : J,"Q": Q, "K":K })
        result_df.to_csv("result2405new/new_heterogeneous_{}_synthetic_ab_j{}q{}k{}_100_{}.csv".format(str(int(time.time())),str(J), str(Q), str(K), str(uplift_ratio).replace('.','')))
        plt.figure() 
        sns.kdeplot(np.array(debias_B_true) /  np.sqrt(np.array(debias_var_B_true)/(int(Q))) , shade = True,color=tencent_blue,label = "Ours(debiased)",alpha=0.1)
        sns.kdeplot(np.array(undebias_B_true) /  np.sqrt(np.array(undebias_var_B_true)/(int(Q))) , shade = True,color='red',label = "Ours(undebiased)",alpha=0.1)
        sns.kdeplot(np.array(dim_B) / np.sqrt(np.array(dim_var_B)), shade = True,color=tencent_orange,label = "DIM",alpha=0.1)
        plt.plot(x, y_standard_normal, color='black', label="Standard Normal", ls='--')
        plt.legend()
        
        plt.savefig("result2406new/new_heterogeneous_{}_synthetic_ab_j{}q{}k{}_density{}.png".format(str(int(time.time())), str(J), str(Q), str(K), str(uplift_ratio).replace('.','')))

Start K = 5, Q = 1200, J = 30
13/13 [==============================] - 0s 406us/step


KeyboardInterrupt: 

In [42]:
res_tempt.shape

(400, 5)